In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import time
import lime


from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report,confusion_matrix

df = pd.read_csv("crash_data_one_hot_encoded_MLP.csv")
df.head()


,CRASH_YEAR,NUM_LANES,SPD_LIM,TRAFFIC_CTRL_Give Way Sign,TRAFFIC_CTRL_Nil,TRAFFIC_CTRL_Points Man,TRAFFIC_CTRL_School Patrol,TRAFFIC_CTRL_Stop Sign,TRAFFIC_CTRL_Traffic Signal,MULTI_VEH_Cyclist(s)+Pedestrian(s) only,...,STREET_LIGHT_Off,STREET_LIGHT_On,STREET_LIGHT_Unknown,WEATHER_A_Fine,WEATHER_A_Heavy Rain,WEATHER_A_Light Rain,WEATHER_A_Mist,WEATHER_A_Snow,WEATHER_A_Unknown,CRASH_SEV_CODE
0,2000,6,50,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0
1,2000,2,100,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2000,3,100,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,2000,2,100,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,2000,2,70,0,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0


In [5]:
df = sklearn.utils.shuffle(df)
X = df.drop("CRASH_SEV_CODE", axis=1).values
y = df["CRASH_SEV_CODE"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=40)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print(X_train.shape); print(X_test.shape)

(641070, 92)
(33741, 92)


In [6]:
mlp = MLPClassifier(hidden_layer_sizes=(92,54,1), activation='relu', solver='adam', max_iter=500,verbose = True)

start = time.time()
mlp.fit(X_train,y_train)
stop = time.time()

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)
print(f"\n Training time: {stop - start}s")
print("\n training data\n")
print(confusion_matrix(y_train,predict_train))
print("\n")
print(classification_report(y_train,predict_train,labels=pd.unique(predict_train)))
print("\n---------------------------------------------------------------------------------------\n")
print("\n test data\n")
print(confusion_matrix(y_test,predict_test))
print("\n")
print(classification_report(y_test,predict_test,labels=pd.unique(predict_test)))

Iteration 1, loss = 0.71814665
Iteration 2, loss = 0.69638344
Iteration 3, loss = 0.69367360
Iteration 4, loss = 0.69208548
Iteration 5, loss = 0.69117508
Iteration 6, loss = 0.69035189
Iteration 7, loss = 0.68958529
Iteration 8, loss = 0.68916463
Iteration 9, loss = 0.68842370
Iteration 10, loss = 0.68811231
Iteration 11, loss = 0.68741556
Iteration 12, loss = 0.68693451
Iteration 13, loss = 0.68652431
Iteration 14, loss = 0.68614609
Iteration 15, loss = 0.68571546
Iteration 16, loss = 0.68530634
Iteration 17, loss = 0.68485379
Iteration 18, loss = 0.68452543
Iteration 19, loss = 0.68419695
Iteration 20, loss = 0.68377660
Iteration 21, loss = 0.68366363
Iteration 22, loss = 0.68337385
Iteration 23, loss = 0.68318855
Iteration 24, loss = 0.68277196
Iteration 25, loss = 0.68263089
Iteration 26, loss = 0.68228817
Iteration 27, loss = 0.68187987
Iteration 28, loss = 0.68189536
Iteration 29, loss = 0.68143316
Iteration 30, loss = 0.68133388
Iteration 31, loss = 0.68104458
Iteration 32, los

In [8]:
weights =  (mlp.coefs_)
print(weights)


[array([[ 2.58555326e-01, -3.38508608e-01, -1.33150981e+00, ...,
        -4.20730613e-01,  9.59247446e-01,  1.38885037e+00],
       [ 7.81746316e-01,  5.74962594e-01, -3.42836352e-01, ...,
        -5.14344352e-01,  1.14570581e-03, -1.46331579e+00],
       [-1.62340146e-01,  6.89947440e-02, -7.40552627e-01, ...,
        -4.20915930e-01, -9.83637137e-01, -5.31373349e-01],
       ...,
       [ 4.76842959e-01,  2.96876478e-01, -3.14333573e-01, ...,
         2.34409831e-01,  1.69198088e-01,  9.34101083e-02],
       [-6.09238968e-02, -8.28273988e-02,  6.95254806e-02, ...,
        -4.79537252e-02,  1.37001155e-01,  1.30994072e-01],
       [-8.52214293e-01,  8.24132173e-03, -3.65775236e-01, ...,
        -8.05638398e-02, -4.21020716e-01, -1.23235797e-02]]), array([[-3.72909388e-01, -4.71243548e-01,  5.57881703e-01, ...,
        -3.07781825e-01,  5.51285098e-01,  6.89964153e-03],
       [-1.22971922e-01, -2.14933518e-01, -2.17075243e-01, ...,
         1.37091290e-01,  9.36103145e-02,  3.79903712

In [28]:
from sklearn.linear_model import Perceptron
clf = Perceptron(tol=1e-3, random_state=40, verbose=True,n_jobs = -1)
clf.fit(X, y)

coeffs = clf.coef_

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 16288.95, NNZs: 85, Bias: -1.000000, T: 674811, Avg. loss: 36266.002777
Total training time: 0.24 seconds.
-- Epoch 2
Norm: 20229.79, NNZs: 88, Bias: -4.000000, T: 674811, Avg. loss: 209171.019945
Total training time: 0.24 seconds.
-- Epoch 2
Norm: 27231.27, NNZs: 90, Bias: -21.000000, T: 674811, Avg. loss: 698633.960251
Total training time: 0.27 seconds.
-- Epoch 2
Norm: 37119.08, NNZs: 90, Bias: 20.000000, T: 674811, Avg. loss: 814152.820191
Total training time: 0.28 seconds.
-- Epoch 2
Norm: 31386.51, NNZs: 86, Bias: 0.000000, T: 1349622, Avg. loss: 35857.828555
Total training time: 0.49 seconds.
-- Epoch 3
Norm: 22412.37, NNZs: 88, Bias: -8.000000, T: 1349622, Avg. loss: 208710.575692
Total training time: 0.51 seconds.
-- Epoch 3
Norm: 51327.20, NNZs: 91, Bias: -42.000000, T: 1349622, Avg. loss: 696286.148209
Total training time: 0.54 seconds.
-- Epoch 3
Norm: 69654.90, NNZs: 90, Bias: 40.000000, T: 1349622, Avg. loss: 806324.295438

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    9.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    9.1s finished


In [36]:
features = df.columns
minV = 0
maxV = 0
minPlace = 0
maxPlace = 0
count = 0
for i in coeffs:
    for j in i:
        if count == 0:
            minPlace = 0
            maxPlace = 0
            minV = j
            maxV = j
        else:
            if minV > j:
                minV =j
                minPlace = count
            if maxV < j:
                maxV = j
                maxPlace = count
        count+=1
    
    count = 0
    print("-------------------")
    print(" min: ",minV," feature: ",features[minPlace],"\n max: ",maxV," feature: ",features[maxPlace])
    print("-------------------")




-------------------
 min:  -4847.0  feature:  NUM_LANES 
 max:  17190.0  feature:  SPD_LIM
-------------------
-------------------
 min:  -376926.0  feature:  MULTI_VEH_Multi vehicle 
 max:  211287.0  feature:  MULTI_VEH_Vehicle(s)+Pedestrian(s)
-------------------
-------------------
 min:  -349098.0  feature:  MULTI_VEH_Vehicle(s)+Pedestrian(s) 
 max:  472311.0  feature:  MULTI_VEH_Multi vehicle
-------------------
-------------------
 min:  -131925.0  feature:  MULTI_VEH_Multi vehicle 
 max:  98814.0  feature:  MULTI_VEH_Vehicle(s)+Pedestrian(s)
-------------------
